# Frame Sequence Extractor

Extracts frame sequences from the dataset.

### Preparation

Place the h2odataset in the following directory relative to the workspace:
`data/h2o/h2odataset`

### Run

- Import the required packages and set the required parameters
- Select the required mode by running the corresponding cell (only)
- Follow along the cells in a chapter to extract the corresponding data

### Output

- The extracted data is saved in numpy's .npy files in `data/h2o` in corresponding folders. 
- For example, the 8 frame sequences for training are stored int `data/h2o/frame_seq_8_train/`.
- Images are stored in the format HxWxC in BGR format (TODO verify :D). 
- The name of the files corresponds to the id provided in the action files of the dataset (e.g. 1.npy).

---

#### Imports

In [2]:
import os
import cv2
import numpy as np
from numpy.typing import NDArray
import pandas as pd
from tqdm import tqdm

#### Parameters

In [3]:
n_frames_per_seq : int = 8
h2o_root = '../data/h2o/'
h2odataset_root = '../data/h2o/h2odataset/'
h2o_actions = h2odataset_root + 'action_labels/'

## Choose Mode

> (Only) Run the cell corresponding to the requested mode (train, val).

The consequent extractions automatically extract data of the configured mode.

Training Set

In [4]:
mode = 'train'

df = pd.read_csv(h2o_actions + 'action_train.txt', delimiter=' ')
ids : list[int] = df['id'].to_list()
paths : list[str] = df['path'].to_list()
labels : list[int] = df['action_label'].to_list()
start_act : list[int] = df['start_act'].to_list()
end_act : list[int] = df['end_act'].to_list()

Validation Set

In [16]:
mode = 'val'

df = pd.read_csv(h2o_actions + 'action_val.txt', delimiter=' ')
ids : list[int] = df['id'].to_list()
paths : list[str] = df['path'].to_list()
labels : list[int] = df['action_label'].to_list()
start_act : list[int] = df['start_act'].to_list()
end_act : list[int] = df['end_act'].to_list()

## Action Labels

In [18]:
path = h2o_root + 'action_labels_' + mode + '.npy'
action_labels = np.array(labels)
np.save(path, action_labels)

## Frame Sequences

Helper Functions

In [5]:
def check_img_sample(img_path: str):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    print("Shape:", img.shape)
    print("dtype:", img.dtype)

    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
def sample_frame_sequence(path: str, start_act: int, end_act: int, n_frames_per_seq: int) -> NDArray:
    """
    Sample the requested amount of frames from an action sequence in the dataset.

    Args:
    -----
        path (str): path to the action sequence images
        start_act (int): starting frame of the action
        end_act (int): ending frame of the action
        n_frames_per_seq (int): number of frames to sample

    Returns:
    --------
        NDArray: array of the sampled frame sequence
    """
    
    n_frames_total = end_act-start_act+1 # total number of frames of the action in the dataset
    assert n_frames_total >= n_frames_per_seq, f"Requested {n_frames_per_seq} frames, but action has only {n_frames_total} frames"

    frame_ids = np.linspace(start_act, end_act, n_frames_per_seq, dtype=int)

    frames : list[NDArray] = []
    for i in range(n_frames_per_seq):
        img_path = path + f'/{frame_ids[i]:05d}.jpg' # TODO how many digits?
        img : NDArray = cv2.imread(img_path, cv2.IMREAD_COLOR)
        
        # TODO any conversions necessary? (RGB, uint8, ...)

        frames.append(img)

    return np.array(frames)

#### Prepare Extraction

Check whether the output image, as well as shape and type are correct.

In [7]:
dest_path = h2o_root + f'frame_seq_{n_frames_per_seq}_{mode}/' # TODO
# check if destination folder exists, if not create it
if not os.path.exists(dest_path):
    os.makedirs(dest_path)

check_img_sample(h2odataset_root + 'obj_point_order.jpg')

Shape: (1552, 1909, 3)
dtype: uint8


#### Extraction

In [ ]:
for i in tqdm(range(len(ids))):
    id : int = ids[i]
    path : str = paths[i]
    # TODO what is the root of the image data?
    frames = sample_frame_sequence(h2odataset_root + path, start_act[i], end_act[i], n_frames_per_seq)
    np.save(dest_path + f'{ids[i]}.npy', frames)

## Pose Sequences